In [ ]:
import glob
import os
import re
from html.parser import HTMLParser
import gensim as gs
from gensim.models import word2vec
import nltk
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns; sns.set()
import csv


stopwords = set(stopwords.words('english'))

path = os.getcwd()

basic_dict = {}
bm_dict = {}
clean_text_dict ={}
model_run_dict = {}
year_list = []
model_train_dict = {}
final_number_dict = {}
company_count_dict = {}

for directory in glob.glob(path+ "/" + "*"):
    split_word_list = []
    if directory[-1] == "b":
        pass
    else:
        year = directory[-4:]
        basic_dict[year] = {}
        bm_dict[year] = {}
        clean_text_dict[year] = {}
        model_run_dict[year] = {}
        final_number_dict[year] = {}
        company_count_dict[year] = {}
        year_list.append(year)
        dir_path = path + "/" + year
        for company in glob.glob(dir_path + "/*.txt"):
            html = HTMLParser()
            company_code = company.split('/')[-1].split('.')[0]
            data = open(company, 'rt').read()
            data = re.sub('<[^>]*>', '', data)
            data = html.unescape(data).encode('ascii', 'ignore').decode('ascii')  
            basic_dict[year][company_code] = data
        for key in basic_dict[year].keys():
            bm_dict[year][key] = " "
            text = basic_dict[year][key]
            bmodel_re = re.compile(r'.*business\s+model.*\n', re.I | re.M | re.U)
            bmodel = re.findall(bmodel_re, str(text))
            if len(bmodel) == 1:
                pos = text.index(" ".join(bmodel))
                before = pos - 500 + len(bmodel)
                after = pos + 500
                beforep = text[before:pos].strip()
                afterp = text[pos:after].strip()
                totalp = beforep + " " + afterp
                totalp = totalp
                bm_dict[year][key] += totalp
            if len(bmodel) > 1:
                multiple_match_list = []
                for string in bmodel:
                    pos = text.index(string)
                    before = pos - 500 + len(string)
                    after = pos + 500
                    beforep = text[before:pos].strip()
                    afterp = text[pos:after].strip()
                    totalp = beforep + " " + afterp
                    totalp = totalp
                    multiple_match_list.append(totalp)
                bm_dict[year][key] = multiple_match_list
            else:
                pass
        for key in bm_dict[year].keys():
            try:
                non_stopword_list = []
                split_text = bm_dict[year][key].lower().split()
                for word in split_text:
                    if word not in stopwords:
                        non_stopword_list.append(word)
                if non_stopword_list != []:
                    split_word_list.append(non_stopword_list)
                    clean_text_dict[year][key] = non_stopword_list
            except AttributeError:
                non_stopword_list = []
                for line in bm_dict[year][key]:
                    split_text = line.lower().split()
                    for word in split_text:
                        if word not in stopwords:
                            non_stopword_list.append(word)
                    if non_stopword_list != []:
                        split_word_list.append(non_stopword_list)
                        clean_text_dict[year][key] = non_stopword_list
        model_run_dict[year] = split_word_list
        print(year + ":" + "Complete")
        
for year in year_list:
    if year == '2013':
        pass
    else:
        final_number_dict[year] = gs.models.Word2Vec(model_run_dict[year], min_count=1, sg=1, seed=1)
        predict_output_dict[year] = final_number_dict[year].predict_output_word(['model'], topn=10)
        
with open('Top10BM.csv','w') as out:
    csv_out=csv.writer(out)
    for year in year_list:
        if year == "2013":
            pass
        else:
            for key, value in predict_output_dict[year]:
                line = [year, key, value]
                csv_out.writerow(line)